# 1. Import the necessary libraries for you to be able to query 4square API

In [1]:
import requests
import json
import pandas as pd
from getpass import getpass

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
token = os.getenv("token")

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [50]:
def requests_for_foursquare (query, lat, lon, radius=1000, limit=10):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}&sort=DISTANCE"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

In [51]:
lat_ih = 41.3977461
lon_ih = 2.187674417
coffee_shops = requests_for_foursquare ("coffee", lat_ih, lon_ih, radius=500, limit=10)

In [54]:
names = [(coffee_shops["results"][i]["name"]) for i in range(10)]
location = [(coffee_shops["results"][i]["location"]["address"]) for i in range(10)]

new_dict = {}
new_dict["name"] = names
new_dict["location"] = location
coffee_shops = pd.DataFrame(new_dict)
coffee_shops

,name,location
0,Espacio 88,"Carrer Pamplona, 88"
1,SKYE Coffee Co,"Calle Pamplona, 88"
2,Granja Mabel,"Marina, 114-116"
3,Three Marks Coffee,"Ausiàs Marc, 151"
4,El Bar,"Ribes, 46"
5,El Cafè Blau,"Casp, 192"
6,Cantina Restobar,"Marina, 164"
7,Eva-2,"Bolivia, 21"
8,De Pa i Xocolata,"Carrer de la Marina, 92"
9,La Rourera,"Gran Vía de Les Corts Catalanes, 792"


# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [36]:
res = requests_for_foursquare ("coffee", lat_ih, lon_ih, radius=500, limit=5)

In [37]:
names = [(res["results"][i]["name"]) for i in range(5)]
distance = [(res["results"][i]["distance"]) for i in range(5)]

new_dict = {}
new_dict["name"] = names
new_dict["distance"] = distance
df = pd.DataFrame(new_dict)
df

,name,distance
0,Espacio 88,311
1,SKYE Coffee Co,320
2,Granja Mabel,327
3,Three Marks Coffee,354
4,El Bar,364


# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [47]:
def foursquare_places (venue, coordinates):
    
    url = f"https://api.foursquare.com/v3/places/search?query={venue}&ll={coordinates[0]}%2C{coordinates[1]}&radius=10000&limit={limit}&sort=DISTANCE"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        res = requests.get(url, headers=headers).json()
        names = [res["results"][i]["name"] for i in range(limit)]
        lat = [res["results"][i]["geocodes"]["main"]["latitude"] for i in range(limit)]
        lon = [res["results"][i]["geocodes"]["main"]["longitude"] for i in range(limit)]

        db = {}

        db["name"] = names
        db["latitude"] = lat
        db["longitud"] = lon

        df = pd.DataFrame(db)
        return df
        
    except:
        print("no :(")


In [48]:
venue = "starbucks"
coordinates = [41.3977461, 2.187674417]
res = foursquare_places(venue, coordinates)
limit = 5
res

,name,latitude,longitud
0,Starbucks,41.392007,2.180638
1,Starbucks,41.404985,2.192439
2,Starbucks,41.402337,2.174068
3,Starbucks,41.388819,2.171884
4,Starbucks,41.385209,2.177359


# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [56]:
res.to_json("data/coffe_shops.json")

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [68]:
res.to_csv("data/res.csv", index=False)

In [70]:
import pymongo
client = pymongo.MongoClient("localhost:27017")
db = client["Ironhack"]
c = db.get_collection("ironcoffee")

data = pd.read_csv("data/res.csv").to_dict(orient="records")

# Insert the data into the MongoDB collection
c.insert_many(data)

InsertManyResult([ObjectId('6543d974360b9a2873f733d0'), ObjectId('6543d974360b9a2873f733d1'), ObjectId('6543d974360b9a2873f733d2'), ObjectId('6543d974360b9a2873f733d3'), ObjectId('6543d974360b9a2873f733d4')], acknowledged=True)